![](img\Architecture.jpg)

In [1]:
#Librerías

%pip install pandas requests beautifulsoup4 langchain-mongodb pymongo python-dotenv langchain-openai langchain-nomic

import pandas as pd
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_community.document_loaders.mongodb import MongodbLoader
import nest_asyncio
from pymongo.mongo_client import MongoClient
import os
from dotenv import load_dotenv
from langchain_nomic import NomicEmbeddings
import pymongo
import json

Note: you may need to restart the kernel to use updated packages.


In [2]:
load_dotenv()
os.environ["USER_AGENT"] = "MyApp/1.0"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
termino_de_busqueda = input('Ingrese el término de búsqueda: ')
termino_de_busqueda = termino_de_busqueda.replace(' ', '+')


In [4]:
# Definir el encabezado con el User-Agent
from bs4 import BeautifulSoup
import requests

#Construcción url
URL = 'https://www.corteconstitucional.gov.co/relatoria/buscador_new/?searchOption=texto&fini=1992-01-01&ffin=2024-10-29&buscar_por='+ termino_de_busqueda +'&accion=search&verform=si&slop=1&volver_a=relatoria&qu=625&maxprov=100&OrderbyOption=des__score'

# Realizar la solicitud GET a la página
response = requests.get(URL)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar todas las etiquetas 'a' con atributo 'href'
    enlaces = [a['href'] for a in soup.find_all('a', href=True)]
else:
    print(f"Error al acceder a la página: {response.status_code}")

# crear una lista con los enlaces obtenidos anteriormente

lista_enlaces = []

# Encontrar todas las etiquetas 'a' con atributo 'href'
for a in soup.find_all('a', href=True):
  lista_enlaces.append(a['href'])


# filtrar lista_enlaces y coger solo los enlaces contengan la palabra relatoria

enlaces_relatoria = [enlace for enlace in lista_enlaces if 'relatoria' in enlace]
enlaces_relatoria = [enlace for enlace in enlaces_relatoria if len(enlace) > 49]

# Imprimir la lista de enlaces filtrados
enlaces_relatoria

['https://www.corteconstitucional.gov.co/relatoria/2024/T-394-24.htm']

In [5]:
# diccionario_relatorias
diccionario_relatorias = {}

for enlace in enlaces_relatoria:
    try:
        nota = requests.get(enlace)
        nota.raise_for_status()  # Verifica si hubo algún problema con la respuesta HTTP
        s_nota = BeautifulSoup(nota.text, 'html.parser')
        texto = (s_nota.find('div', attrs={'class': 'WordSection1'}).text).strip()
        diccionario_relatorias[enlace] = texto
    except requests.exceptions.RequestException as e:
        print(f"Error al solicitar el enlace {enlace}: {e}")
    except AttributeError:
        try:
            # Si no encuentra 'WordSection1', intenta con 'Section1'
            texto = (s_nota.find('div', attrs={'class': 'Section1'}).text).strip()
            diccionario_relatorias[enlace] = texto
        except AttributeError as e:
            print(f"Error procesando el contenido del enlace {enlace}: {e}")

diccionario_relatorias

{'https://www.corteconstitucional.gov.co/relatoria/2024/T-394-24.htm': 'REPÚBLICA DE COLOMBIA\n\xa0\n\n\xa0\nCORTE CONSTITUCIONAL\nSala Cuarta de Revisión\n\xa0\nSENTENCIA T-394 DE 2024\n\xa0\n\xa0\nReferencia:\nexpediente T-9.359.163\n\xa0\nRevisión de las decisiones judiciales relacionadas con\r\nla solicitud de tutela presentada por Mónica\r\nMaría Restrepo Palacios, representante legal de la sociedad El Colombiano\r\nS.A.S., contra Daniel Quintero Calle\n\xa0\nMagistrado sustanciador:\nANTONIO JOSÉ LIZARAZO OCAMPO\n\xa0\n\xa0\n\n\n\n\nSíntesis de la decisión. La Sala Cuarta de Revisión declaró improcedente la solicitud de\r\n  tutela presentada por la representante legal de El Colombiano contra el\r\n  entonces alcalde de Medellín, Daniel Quintero Calle, porque no cumplió el\r\n  requisito de subsidiariedad. Al respecto, advirtió que, a pesar de que la\r\n  accionante no pretendía que el alcalde rectificara o eliminara los mensajes\r\n  publicados en su cuenta de Twitter relacionad

In [6]:
# convertir el diccionario en un df de pandas

df = pd.DataFrame(list(diccionario_relatorias.items()), columns=['Enlace', 'Texto'])

df['Sentencia'] = df['Enlace'].str.split('/relatoria/').str[-1].str.split('.htm').str[0]

# Reorganizar el DataFrame
df = df[['Sentencia', 'Texto']]  # Selecciona las columnas en el orden deseado

In [7]:
# exportar el df en formato JSON Lines

nombre_json = ('sentencias_' + termino_de_busqueda).replace('+', '_') + '.jsonl'
# Assuming df is your DataFrame
df.to_json(nombre_json, orient='records', lines=True)

In [8]:
client = pymongo.MongoClient(os.environ['MONGODB_URI'])
collections = client.get_database(os.environ['MONGODB_DB']).get_collection(os.environ['MONGODB_COLLECTION'])

In [9]:
collections.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff000000000000015d'), 'opTime': {'ts': Timestamp(1732117331, 7), 't': 349}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1732117331, 7), 'signature': {'hash': b'\x07\xa3\xafE\x17e>_\xa8@{\x19\xcc\x11\x9f\xfc\x04y\x83\x8e', 'keyId': 7400451993602359319}}, 'operationTime': Timestamp(1732117331, 7)}, acknowledged=True)

In [10]:
# Leer el archivo JSON Lines y cargar los documentos
with open(nombre_json, 'r') as f:
    docs_to_insert = [json.loads(line) for line in f]

In [11]:
%pip install langchain

from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

docs_to_insert = [
    Document(page_content=doc['Texto'], metadata={'sentencia': doc['Sentencia']})
    for doc in docs_to_insert
]

def custom_split(text, max_size=1000):
    chunks = []
    while len(text) > max_size:
        chunk = text[:max_size]
        chunks.append(chunk)
        text = text[max_size:]
    if text:
        chunks.append(text)
    return chunks

docs_splits = []
for doc in docs_to_insert:
    chunks = custom_split(doc.page_content)
    for chunk in chunks:
        docs_splits.append(Document(page_content=chunk, metadata=doc.metadata))
docs_splits_dict = [doc.dict() for doc in docs_splits]

collections.insert_many(docs_splits_dict)


Note: you may need to restart the kernel to use updated packages.


InsertManyResult([ObjectId('673e035e0dca7c202b9033dc'), ObjectId('673e035e0dca7c202b9033dd'), ObjectId('673e035e0dca7c202b9033de'), ObjectId('673e035e0dca7c202b9033df'), ObjectId('673e035e0dca7c202b9033e0'), ObjectId('673e035e0dca7c202b9033e1'), ObjectId('673e035e0dca7c202b9033e2'), ObjectId('673e035e0dca7c202b9033e3'), ObjectId('673e035e0dca7c202b9033e4'), ObjectId('673e035e0dca7c202b9033e5'), ObjectId('673e035e0dca7c202b9033e6'), ObjectId('673e035e0dca7c202b9033e7'), ObjectId('673e035e0dca7c202b9033e8'), ObjectId('673e035e0dca7c202b9033e9'), ObjectId('673e035e0dca7c202b9033ea'), ObjectId('673e035e0dca7c202b9033eb'), ObjectId('673e035e0dca7c202b9033ec'), ObjectId('673e035e0dca7c202b9033ed'), ObjectId('673e035e0dca7c202b9033ee'), ObjectId('673e035e0dca7c202b9033ef'), ObjectId('673e035e0dca7c202b9033f0'), ObjectId('673e035e0dca7c202b9033f1'), ObjectId('673e035e0dca7c202b9033f2'), ObjectId('673e035e0dca7c202b9033f3'), ObjectId('673e035e0dca7c202b9033f4'), ObjectId('673e035e0dca7c202b9033

In [12]:
%pip install motor
nest_asyncio.apply()
load_dotenv()
loader = MongodbLoader(
    connection_string=os.environ['MONGODB_URI'],
    db_name=os.environ['MONGODB_DB'],
    collection_name=os.environ['MONGODB_COLLECTION'],
    filter_criteria={},
    field_names=["metadata", "page_content"],
)
docs = loader.load()

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

C:\Users\olart\AppData\Local\Temp\ipykernel_3200\1010149463.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [14]:
vectorStore = MongoDBAtlasVectorSearch.from_documents( 
    documents= docs,
    embedding= embeddings, 
    collection= collections,
    index_name=os.environ['MONGODB_VECTOR_INDEX']
)

In [15]:
retriever = vectorStore.as_retriever(search_kwargs={"similarity_threshold": 0.1})

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

template = """Quiero un análisis jurídico profesional sobre los derechos fundamentales amenazados o dañados que se debaten en la Corte Constitucional de Colombia. Es importante conocer los hechos de acuerdo a las circunstancias de modo, el tiempo con las fechas y hora, el lugar
donde ocurrieron y las personas naturales o jurídicas que tienen conflicto entre ellas. También es importante conocer cuál fue el daño o peligro que afecta los derechos fundamentales dentro de las consideraciones tenidas en cuenta por la Corte Constitucional. Finalmente, requiero saber lo que resuelve la Corte Constitucional. Con base en las anteriores instrucciones, proporciona un resumen de:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0, model="gpt-4o")

# Local LLM
ollama_llm = "phi3.5"
model_local = ChatOllama(model=ollama_llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = """ De la sentencia T-394-24, responde estos 5 puntos:
	1.	Los hechos de la sentencia.
	2.	Los derechos fundamentales en discusión dentro de la sentencia.
	3.	Las consideraciones más relevantes de la Corte Constitucional en su análisis y decisión.
	4.	La conclusión o sentencia emitida por la Corte.
	5.	Cuál es el razonamiento jurídico que utiliza la Corte Constitucional para conceder o negar derechos fundamentales?"""
results = chain.invoke(query)
print(results)

Para proporcionar un análisis detallado de la sentencia T-394-24 de la Corte Constitucional de Colombia, abordaré los cinco puntos solicitados:

1. **Los hechos de la sentencia:**
   La sentencia T-394-24 se centra en un conflicto entre el medio de comunicación "El Colombiano" y el señor Daniel Quintero Calle. El medio de comunicación presentó una acción de tutela solicitando la protección de sus derechos fundamentales, alegando que estos habían sido afectados por publicaciones realizadas por Quintero Calle. La acción de tutela fue presentada debido a que "El Colombiano" consideró que no existía otro medio idóneo y eficaz para proteger sus derechos.

2. **Los derechos fundamentales en discusión dentro de la sentencia:**
   Los derechos fundamentales en discusión incluyen la libertad de expresión, la libertad de prensa, el buen nombre y la no discriminación. "El Colombiano" buscaba el amparo de estos derechos, argumentando que las publicaciones de Quintero Calle afectaban su buen nombre